In [ ]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [ ]:
# get data files
# !wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

# !unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

In [ ]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
df_ratings['rating'] = scaler.fit_transform(df_ratings['rating'].values.reshape(-1, 1))

In [ ]:
# add your code here - consider creating a new cell for each section of code
df = df_ratings

user_counts = df['user'].value_counts()
isbn_counts = df['isbn'].value_counts()

df = df[~df['user'].isin(user_counts[user_counts < 200].index)]
df = df[~df['isbn'].isin(isbn_counts[isbn_counts < 100].index)]

df = pd.merge(right=df, left=df_books, on="isbn")
df = df.drop_duplicates(['user', 'title'])

pivot = df.pivot(index='title', columns='user', values='rating').fillna(0)
matrix = csr_matrix(pivot)

NameError: name 'df_ratings' is not defined

In [ ]:
model = NearestNeighbors(metric='cosine', algorithm='brute', p=2)
model.fit(matrix)

NameError: name 'NearestNeighbors' is not defined

In [ ]:
# function to return recommended books - this will be tested
def get_recommends(book = ""):
  recommended_books = []
  recommended_books.append(book)

  data = csr_matrix(pivot.loc[book].values.reshape(1, -1))
  distance, indicies = model.kneighbors(data, n_neighbors=6)

  knn_books = pivot.iloc[indicies[0]].index.values.tolist()
  recommended_books.append([])

  for item in zip(knn_books, distance[0]):
    recommended_books[1].append([item[0], item[1]])

  recommended_books[1].pop(0)
  recommended_books[1].reverse()

  return recommended_books

In [ ]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()